In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')


In [ ]:
dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
for col in ['sex','smoker']:
  encoder=LabelEncoder()
  dataset[col]=encoder.fit_transform(dataset[col])

dataset=pd.get_dummies(dataset,columns=['region'],drop_first=True)

In [ ]:
dataset

In [ ]:
train_dataset,test_dataset=train_test_split(dataset,test_size=0.2,random_state=42)
train_labels=train_dataset.pop('expenses')
test_labels=test_dataset.pop('expenses')

In [ ]:
scaler=StandardScaler()
train_dataset=pd.DataFrame(scaler.fit_transform(train_dataset),columns=train_dataset.columns)
test_dataset=pd.DataFrame(scaler.transform(test_dataset),columns=test_dataset.columns)

In [ ]:
model=keras.Sequential([
    layers.Dense(64,activation='relu',input_shape=[len(train_dataset.keys())]),
    layers.Dropout(0.2),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.compile(optimizer='adam',loss='mae',metrics=['mae'])

In [ ]:
history=model.fit(
    train_dataset,
    train_labels,
    epochs=70,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
